In [1]:
!pip install datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

# Load the GoEmotions dataset
dataset = load_dataset("go_emotions")

# Simplify the dataset by filtering out examples with multiple labels
def simplify(example):
    return len(example['labels']) == 1

dataset = dataset.filter(simplify)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.40k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/350k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Filter:   0%|          | 0/43410 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5426 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [3]:
# Extract text and single-label categories
def preprocess_labels(example):
    example['label'] = example['labels'][0]  # Take the single label
    return example

dataset = dataset.map(preprocess_labels)
dataset = dataset.remove_columns(['labels'])

Map:   0%|          | 0/36308 [00:00<?, ? examples/s]

Map:   0%|          | 0/4548 [00:00<?, ? examples/s]

Map:   0%|          | 0/4590 [00:00<?, ? examples/s]

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorizing the text data using TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_features=5000,
    min_df=2, max_df=0.95      # Ignore rare and very common words
)
X_train = vectorizer.fit_transform(dataset['train']['text'])
X_val = vectorizer.transform(dataset['validation']['text'])
X_test = vectorizer.transform(dataset['test']['text'])

y_train = dataset['train']['label']
y_val = dataset['validation']['label']
y_test = dataset['test']['label']

In [5]:
# feature scaling
from sklearn.preprocessing import MaxAbsScaler

# Apply MaxAbsScaler to normalize features
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [6]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=100,   # Number of trees in the forest
    random_state=42     # For reproducibility
)

In [ ]:
rf_model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_train_pred_rf = rf_model.predict(X_train_scaled)
y_val_pred_rf = rf_model.predict(X_val_scaled)
y_test_pred_rf = rf_model.predict(X_test_scaled)

train_accuracy_rf = accuracy_score(y_train, y_train_pred_rf)
val_accuracy_rf = accuracy_score(y_val, y_val_pred_rf)
test_accuracy_rf = accuracy_score(y_test, y_test_pred_rf)

print(f"Random Forest Training Accuracy: {train_accuracy_rf:.4f}")
print(f"Random Forest Validation Accuracy: {val_accuracy_rf:.4f}")
print(f"Random Forest Test Accuracy: {test_accuracy_rf:.4f}")

Random Forest Training Accuracy: 0.9934
Random Forest Validation Accuracy: 0.5638
Random Forest Test Accuracy: 0.5503


In [ ]:
from sklearn.model_selection import GridSearchCV
# Define hyperparameter grid
param_grid_rf = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [None, 10, 20, 30], # Depth of each tree
    'min_samples_split': [2, 5, 10]  # Minimum samples to split a node
}

# Perform grid search
grid_search_rf = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid_rf,
    scoring='accuracy',
    cv=3,
    verbose=1
)

grid_search_rf.fit(X_train_scaled, y_train)

# Print best parameters and accuracy
print(f"Best Parameters: {grid_search_rf.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search_rf.best_score_:.4f}")

# Use the best model
best_rf_model = grid_search_rf.best_estimator_


Fitting 3 folds for each of 36 candidates, totalling 108 fits


KeyboardInterrupt: 

In [ ]:
rf_model_2 = RandomForestClassifier(
    max_depth=None,
    min_samples_split=10,
    n_estimators=200,   # Number of trees in the forest
    random_state=42     # For reproducibility
)

In [ ]:
rf_model_2.fit(X_train_scaled, y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=200, random_state=42)

In [ ]:
y_train_pred_rf = rf_model_2.predict(X_train_scaled)
y_val_pred_rf = rf_model_2.predict(X_val_scaled)
y_test_pred_rf = rf_model_2.predict(X_test_scaled)

train_accuracy_rf = accuracy_score(y_train, y_train_pred_rf)
val_accuracy_rf = accuracy_score(y_val, y_val_pred_rf)
test_accuracy_rf = accuracy_score(y_test, y_test_pred_rf)

print(f"Random Forest Training Accuracy: {train_accuracy_rf:.4f}")
print(f"Random Forest Validation Accuracy: {val_accuracy_rf:.4f}")
print(f"Random Forest Test Accuracy: {test_accuracy_rf:.4f}")

Random Forest Training Accuracy: 0.9821
Random Forest Validation Accuracy: 0.5657
Random Forest Test Accuracy: 0.5529


In [ ]:
# Define parameter grid
param_grid_rf = {
    'min_samples_leaf': [1, 2, 4],        # Minimum samples per leaf
    'max_features': ['sqrt', 'log2']     # Number of features considered
}

# Grid search
grid_search_rf = GridSearchCV(
    RandomForestClassifier(random_state=42, max_depth=None, min_samples_split=10, n_estimators=200),
    param_grid_rf,
    scoring='accuracy',
    cv=3,
    verbose=1
)

grid_search_rf.fit(X_train_scaled, y_train)

# Best parameters
print(f"Best Parameters: {grid_search_rf.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search_rf.best_score_:.4f}")

# Evaluate the best model
best_rf_model = grid_search_rf.best_estimator_

In [ ]:
rf_model_3 = RandomForestClassifier(
    max_depth=None,
    min_samples_split=10,
    n_estimators=200,   # Number of trees in the forest
    max_features='sqrt',
    min_samples_leaf=1,
    random_state=42     # For reproducibility
)

In [ ]:
rf_model_3.fit(X_train_scaled, y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=200, random_state=42)

In [ ]:
y_train_pred_rf = rf_model_3.predict(X_train_scaled)
y_val_pred_rf = rf_model_3.predict(X_val_scaled)
y_test_pred_rf = rf_model_3.predict(X_test_scaled)

train_accuracy_rf = accuracy_score(y_train, y_train_pred_rf)
val_accuracy_rf = accuracy_score(y_val, y_val_pred_rf)
test_accuracy_rf = accuracy_score(y_test, y_test_pred_rf)

print(f"Random Forest Training Accuracy: {train_accuracy_rf:.4f}")
print(f"Random Forest Validation Accuracy: {val_accuracy_rf:.4f}")
print(f"Random Forest Test Accuracy: {test_accuracy_rf:.4f}")

Random Forest Training Accuracy: 0.9821
Random Forest Validation Accuracy: 0.5657
Random Forest Test Accuracy: 0.5529


In [7]:
best_rf_model  = RandomForestClassifier(random_state=42, max_depth=None, min_samples_split=10, n_estimators=200, max_features='sqrt', min_samples_leaf=1)
best_rf_model.fit(X_train_scaled, y_train)
# Predict on all datasets
y_train_pred = best_rf_model.predict(X_train_scaled)
y_val_pred = best_rf_model.predict(X_val_scaled)
y_test_pred = best_rf_model.predict(X_test_scaled)

# Calculate accuracies
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print the accuracies
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Detailed classification reports
print("\nClassification Report (Training Set):")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report (Validation Set):")
print(classification_report(y_val, y_val_pred))

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_test_pred))

Training Accuracy: 0.9821
Validation Accuracy: 0.5657
Test Accuracy: 0.5529

Classification Report (Training Set):
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      2710
           1       0.98      0.99      0.98      1652
           2       0.98      0.96      0.97      1025
           3       0.99      0.97      0.98      1451
           4       1.00      0.96      0.98      1873
           5       0.98      0.98      0.98       649
           6       1.00      0.97      0.98       858
           7       1.00      0.96      0.98      1389
           8       0.99      0.99      0.99       389
           9       1.00      0.98      0.99       709
          10       1.00      0.98      0.99      1402
          11       0.98      0.97      0.97       498
          12       1.00      0.99      0.99       203
          13       0.98      0.89      0.94       510
          14       0.99      0.99      0.99       430
          15       0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m